Stage 1: Data crawler. Massively parallel functions crawling data and storing it in Cloud Object Storage.  Obtain information from web pages or tweets and create a dataset of text data. Use FaaS backend in lithops to launch crawling process over serverless functions.

In [1]:
import lithops


In [3]:
from custom_snscrape.twitter import TwitterSearchScraper

# TODO: Improve function names...
# TODO: Read this https://github.com/lithops-cloud/lithops/blob/master/docs/api_storage.md
# TODO: put_object vs put_cloudobject?

def scrapSearch(str_to_search, storage):
    for tweet in TwitterSearchScraper(str_to_search).get_items():

        # key_prefix is only the included hashtag, without the excluded ones
        key_prefix = str_to_search.split()[0]

        storage.put_object(bucket='colab',
                              key=key_prefix+'/'+str(tweet.id),
                              body=tweet.to_json())
        
        break # testing: only 1 result

    return "OK"


with lithops.FunctionExecutor() as fexec:
    # Each query will include the current hashtag,
    # and exclude the hashtags to the right of itself in the array.
    # This way, we can parallelise the searches, while also ensuring that
    # if a tweet contains multiple hashtags, one of the queries includes said tweet.
    # 
    # In this case, the queries will be:
    # #covid -#sars -#coronavirus, #sars -#coronavirus, #coronavirus
    hashtags = ['#covid', '#sars', '#coronavirus'] # TODO: Use a jupyter widget (textbox perhaps?) to insert here the hashtags.
    queries = []

    for i in range(len(hashtags)):
        queries.append(' -'.join(hashtags))
        hashtags.pop(0)

    all_invocations = fexec.map(scrapSearch, queries)
    print(fexec.get_result())

2021-05-22 01:15:46,649 [INFO] lithops.config -- Lithops v2.3.3
2021-05-22 01:15:46,656 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- IBM COS Storage client created - Region: eu-de
2021-05-22 01:15:47,325 [INFO] lithops.serverless.backends.ibm_cf.ibm_cf -- IBM CF client created - Region: eu-de - Namespace: Namespace-n4e-colab
2021-05-22 01:15:47,327 [INFO] lithops.executors -- Serverless Executor created with ID: 73591e-1
2021-05-22 01:15:47,329 [INFO] lithops.invokers -- ExecutorID 73591e-1 | JobID M000 - Selected Runtime: lithopscloud/ibmcf-python-v39 - 256MB
2021-05-22 01:15:47,338 [INFO] lithops.job.job -- ExecutorID 73591e-1 | JobID M000 - Uploading function and data - Total: 943.0B
2021-05-22 01:15:47,925 [INFO] lithops.invokers -- ExecutorID 73591e-1 | JobID M000 - Starting function invocation: scrapSearch() - Total: 3 activations
2021-05-22 01:15:47,929 [INFO] lithops.invokers -- ExecutorID 73591e-1 | JobID M000 - View execution logs at /tmp/lithops/logs/73591e-1-M000.log


ModuleNotFoundError: No module named 'custom_snscrape'

In [ ]:
Tweets: 
1) Pedro Sanchez
 https://twitter.com/sanchezcastejon/status/1388184494235127814
 https://twitter.com/sanchezcastejon/status/1376241393396543492
 https://twitter.com/sanchezcastejon/status/1371013139987365888

2)  Boris Johnson
https://twitter.com/BorisJohnson/status/1391413912889921542
https://twitter.com/BorisJohnson/status/1363552116133023744
https://twitter.com/BorisJohnson/status/1354147835369959435

3) Joe Biden
https://twitter.com/JoeBiden/status/1386426190861197313
https://twitter.com/POTUS/status/1373037818277167112
https://twitter.com/POTUS/status/1384136220284915712
https://twitter.com/POTUS/status/1368993395172139014

4) Elon Musk
https://twitter.com/elonmusk/status/1256240766600409088


